In [28]:
"""
This is an upgraded version of Ceshine's LGBM starter script, simply adding more
average features and weekly average features on it.
"""
from datetime import date, timedelta

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

import xgboost as xgb
import sys

import math
import sklearn.metrics as skl_metrics

from datetime import timedelta
from logging import StreamHandler, DEBUG, Formatter, FileHandler, getLogger

df_train = pd.read_csv(
    '../input/train_1s.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    parse_dates=["date"],
)

df_test = pd.read_csv(
    "../input/test_1s.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
)


df_train_mindate = df_train[['item_nbr','store_nbr','date']].groupby(['item_nbr','store_nbr'])\
    ['date'].min().to_frame('min_date')
    
df_train_mindate.reset_index(inplace=True)



/home/zyp/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
df_train_mindate['date'] = df_train_mindate['min_date']

In [62]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2562153 entries, 0 to 2562152
Data columns (total 5 columns):
date           datetime64[ns]
store_nbr      int64
item_nbr       int64
unit_sales     float64
onpromotion    object
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 97.7+ MB


In [42]:
df_train_mindate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3567 entries, 0 to 3566
Data columns (total 4 columns):
item_nbr     3567 non-null int64
store_nbr    3567 non-null int64
min_date     3567 non-null datetime64[ns]
date         3567 non-null datetime64[ns]
dtypes: datetime64[ns](2), int64(2)
memory usage: 111.5 KB


In [43]:
df_train_p = df_train_mindate.loc[ (df_train_mindate['date'] >'2016-08-10'),]
df_train_m = df_train_mindate.loc[ (df_train_mindate['date'] <='2016-08-10'),]

In [32]:
df_train_p.head()

,item_nbr,store_nbr,min_date
1000,849098,1,2016-11-04
1060,876224,1,2016-11-07
1061,876225,1,2016-12-27
1062,876226,1,2016-12-28
1104,906980,1,2017-08-15


In [44]:
df_train_m.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3210 entries, 0 to 3400
Data columns (total 4 columns):
item_nbr     3210 non-null int64
store_nbr    3210 non-null int64
min_date     3210 non-null datetime64[ns]
date         3210 non-null datetime64[ns]
dtypes: datetime64[ns](2), int64(2)
memory usage: 125.4 KB


In [24]:

test_store_item = df_test[['item_nbr','store_nbr','date']].groupby(['item_nbr','store_nbr'])\
    ['date'].count().to_frame('cnt_date')
test_store_item.reset_index(inplace=True)

df_train_new = pd.merge(df_train,test_store_item,how = 'inner',on=['store_nbr','item_nbr'])


In [49]:
train_m = pd.merge(df_train_new,df_train_m,how = 'inner',on=['store_nbr','item_nbr','date'])
train_p = pd.merge(df_train_new,df_train_p,how = 'inner',on=['store_nbr','item_nbr','date'])


In [51]:
train_m[['store_nbr', 'item_nbr', 'date', 'unit_sales','onpromotion']].to_pickle('../input/train_m.p')
train_p[['store_nbr', 'item_nbr', 'date', 'unit_sales','onpromotion']].to_pickle('../input/train_p.p')

In [58]:
train_m_store_item = train_m[['item_nbr','store_nbr','date']].groupby(['item_nbr','store_nbr'])\
    ['date'].count().to_frame('cnt_date')
train_m_store_item.reset_index(inplace=True)
test_m = pd.merge(df_test,train_m_store_item,how = 'inner',on=['store_nbr','item_nbr'])
print("mature  store/items in test", test_m.info())



<class 'pandas.core.frame.DataFrame'>
Int64Index: 1386 entries, 0 to 1385
Data columns (total 6 columns):
id             1386 non-null int64
date           1386 non-null datetime64[ns]
store_nbr      1386 non-null int64
item_nbr       1386 non-null int64
onpromotion    1386 non-null bool
cnt_date       1386 non-null int64
dtypes: bool(1), datetime64[ns](1), int64(4)
memory usage: 66.3 KB
mature  store/items in test None


In [60]:
train_p_store_item = train_p[['item_nbr','store_nbr','date']].groupby(['item_nbr','store_nbr'])\
    ['date'].count().to_frame('cnt_date')
train_p_store_item.reset_index(inplace=True)
test_p= pd.merge(df_test,train_p_store_item,how = 'inner',on=['store_nbr','item_nbr'])
print("partial store/items in test", test_p.info())



<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 0 to 6
Data columns (total 6 columns):
id             7 non-null int64
date           7 non-null datetime64[ns]
store_nbr      7 non-null int64
item_nbr       7 non-null int64
onpromotion    7 non-null bool
cnt_date       7 non-null int64
dtypes: bool(1), datetime64[ns](1), int64(4)
memory usage: 343.0 bytes
partial store/items in test None


In [61]:
test_m[['id','store_nbr', 'item_nbr', 'date', 'onpromotion']].to_pickle('../input/test_m.p')
test_p[['id','store_nbr', 'item_nbr', 'date', 'onpromotion']].to_pickle('../input/test_p.p')